In [ ]:
# | default_exp apps.whoami.domain

In [ ]:
# | export
import ast
from meme_games.core import *
from meme_games.domain import *

In [ ]:
# | export

@dataclass
class PlayerLabel:
    x: int = 0
    y: int = 0
    width: int = 0
    height: int = 0


@dataclass
class WhoAmIPlayer(LobbyMember):
    _lobby_type = 'whoami'
    label_text: str = ''
    label_tfm: Optional[PlayerLabel] = None
    notes: str = ''

    def __post_init__(self):
        if isinstance(self.label_tfm, str): self.label_tfm = PlayerLabel(**ast.literal_eval(self.label_tfm))
        return super().__post_init__()

    def set_notes(self, notes: str): self.notes = notes
    def set_label(self, label: str): self.label_text = label
    def set_label_transform(self, tfm: dict = None): self.label_tfm = PlayerLabel(**(tfm or {}))

    @classmethod
    def columns(cls):
        cols = dict(set(super().columns().items()) - set(LobbyMember.columns().items()))
        cols.update({'id': str})
        return cols

    @classmethod
    def from_dict(cls, data: dict):
        data = cols2dict(data)
        data[cls]['user'] = User.from_dict(data.pop(User))
        data[cls].update(data.pop(LobbyMember))
        return super(LobbyMember, cls).from_dict(data[cls])

WAILobby = Lobby[WhoAmIPlayer]

In [ ]:
db = init_db()
um = UserManager(db)
lm = LobbyManager(db)
mm = register_manager(MemberManager(um), LobbyMember)

In [ ]:
p = WhoAmIPlayer(um.create(1,1), label_text='cat')
p.set_label_transform(dict(x=1,y=1, width=1,height=2))
asdict(p)

{'user_uid': 1,
 'lobby_id': None,
 'is_player': False,
 'is_host_': False,
 'joined_at': datetime.datetime(2025, 1, 19, 16, 11, 25, 182315),
 'score': 0,
 'id': 'jqvawcts',
 'label_text': 'cat',
 'label_tfm': {'x': 1, 'y': 1, 'width': 1, 'height': 2},
 'notes': ''}

In [ ]:
#| export
class WhoAmIManager(DataManager[WhoAmIPlayer]):
    
    def __init__(self, member_manager: MemberManager):
        self.mm = member_manager
        self.mem_t = self.mm.members
        super().__init__(self.mm.db)
    
    def _set_tables(self):
        self.players = self.db.t.whoami_members.create(**WhoAmIPlayer.columns(), 
                                                       pk='id', transform=True, if_not_exists=True,
                                                       foreign_keys=[('id', 'members', 'id')])
        return self.players

    def upsert(self, obj):
        self.mm.upsert(obj._asdict(LobbyMember.columns().keys()))
        return super().upsert(obj._asdict(WhoAmIPlayer.columns().keys()))
    
    def upsert_all(self, objs):
        self.mm.upsert_all([o._asdict(LobbyMember.columns().keys()) for o in objs])
        return super().upsert_all([o._asdict(WhoAmIPlayer.columns().keys()) for o in objs])
    
    def update(self, obj):
        self.mm.update(obj._asdict(LobbyMember.columns().keys()))
        return super().update(obj._asdict(WhoAmIPlayer.columns().keys()))

    def insert(self, obj: WhoAmIPlayer):
        self.mm.insert(obj._asdict(LobbyMember.columns().keys()))
        return super().insert(obj._asdict(WhoAmIPlayer.columns().keys()))
    
    def get_all(self, lobby_id: str) -> list[WhoAmIPlayer]: 
        qry = f'''select {mk_aliases(WhoAmIPlayer, self.players)},  
                 {mk_aliases(LobbyMember, self.mem_t)},
                 {mk_aliases(User, self.mm.users)}
                  from {self.players} \
                  join {self.mem_t} on {self.mem_t.c.id} = {self.players.c.id} \
                  join {self.mm.users} on {self.mem_t.c.user_uid} = {self.mm.users.c.uid} \
                  where {self.mem_t.c.lobby_id} = ?'''
        return list(map(WhoAmIPlayer.from_dict, self.db.q(qry, [lobby_id])))

In [ ]:
#| export
DI.register_service(WhoAmIManager)

In [ ]:
wm = register_manager(WhoAmIManager(mm), WhoAmIPlayer)
l = lm.insert(Lobby(lobby_type=WhoAmIPlayer._lobby_type))
m = l.create_member(um.create())
l.set_host(m)
l.add_member(p)
lm.update(l)

Lobby(id='1zgw1', lobby_type='whoami', host=WhoAmIPlayer(user=User(uid='g0w2c', name='null', filename=UNSET), user_uid='g0w2c', lobby_id='1zgw1', is_player=False, is_host_=True, joined_at=datetime.datetime(2025, 1, 19, 16, 11, 25, 209632), score=0, id='3ysivfkc', send=None, ws=None, label_text='', label_tfm=None, notes=''), members={'g0w2c': WhoAmIPlayer(user=User(uid='g0w2c', name='null', filename=UNSET), user_uid='g0w2c', lobby_id='1zgw1', is_player=False, is_host_=True, joined_at=datetime.datetime(2025, 1, 19, 16, 11, 25, 209632), score=0, id='3ysivfkc', send=None, ws=None, label_text='', label_tfm=None, notes=''), 1: WhoAmIPlayer(user=User(uid=1, name=1, filename=UNSET), user_uid=1, lobby_id='1zgw1', is_player=False, is_host_=False, joined_at=datetime.datetime(2025, 1, 19, 16, 11, 25, 182315), score=0, id='jqvawcts', send=None, ws=None, label_text='cat', label_tfm=PlayerLabel(x=1, y=1, width=1, height=2), notes='')}, last_active=datetime.datetime(2025, 1, 19, 16, 11, 25, 209632))

In [ ]:
lm.get(l.id).members

{'g0w2c': WhoAmIPlayer(user=User(uid='g0w2c', name='null', filename=None), user_uid='g0w2c', lobby_id=None, is_player=False, is_host_=False, joined_at=datetime.datetime(2025, 1, 19, 16, 12, 3, 813565), score=0, id='3ysivfkc', send=None, ws=None, label_text='', label_tfm=None, notes=''),
 '1': WhoAmIPlayer(user=User(uid='1', name='1', filename=None), user_uid='1', lobby_id=None, is_player=False, is_host_=False, joined_at=datetime.datetime(2025, 1, 19, 16, 12, 3, 813565), score=0, id='jqvawcts', send=None, ws=None, label_text='cat', label_tfm=PlayerLabel(x=1, y=1, width=1, height=2), notes='')}